In [3]:
example = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
""".strip().splitlines()

with open("day07.txt", "r") as f:
    data = f.readlines()

In [4]:
from typing import Dict
from re import match, findall

class Bag(object):
    def __init__(self, name: str, contents: Dict[str, int]):
        self.name = name
        self.contents = contents

    @staticmethod
    def from_spec(spec: str) -> "Bag":
        spec_match = match(r"^([\w\s]+) bags contain (.*).$", spec)
        name = spec_match[1]
        contents = {
            item[1]: int(item[0])
            for item in findall(r"(\d+) ([\w\s]+) bags?", spec_match[2])
        }

        return Bag(name, contents)
    

test_bag = Bag.from_spec("light red bags contain 1 bright white bag, 2 muted yellow bags.")
assert test_bag.name == "light red"
assert test_bag.contents == { "bright white": 1, "muted yellow": 2 }

example_bags = [Bag.from_spec(spec) for spec in example]


## Data Structure
We can represent the bags as a graph - in which every node has zero or more parent nodes (these are bags which contain them)
and zero or more child nodes (these are bags that they contain). We can then maintain an index into this graph to allow us to
rapidly locate and traverse between nodes - enabling `O(N)` answers to both the "parents" question (part 1) and the "children"
question (part 2).

In [20]:
from typing import List, Iterator

class BagGraphNode(object):
    def __init__(self, bag: Bag):
        self.bag = bag
        self.parents: List[BagTreeNode] = []
        self.children: Dict[str, BagGraphNode] = {}

class BagGraph(object):
    def __init__(self, bags: List[Bag]):
        self.node_lookup: Dict[str, BagGraphNode] = {}
        
        for bag in bags:
            self.node_lookup[bag.name] = BagGraphNode(bag)

        for bag in bags:
            for child_bag, count in bag.contents.items():
                self.node_lookup[bag.name].children[child_bag] = self.node_lookup[child_bag]
                self.node_lookup[child_bag].parents.append(self.node_lookup[bag.name])

    def get_parents(self, name: str) -> Iterator[str]:
        bag_node = self.node_lookup[name]
        for parent in bag_node.parents:
            yield parent.bag.name

            for rec_parent in self.get_parents(parent.bag.name):
                yield rec_parent

    def get_child_count(self, name: str) -> int:
        total_bags = 0

        bag_node = self.node_lookup[name]
        for child, child_node in bag_node.children.items():
            child_count = bag_node.bag.contents[child]

            total_bags += child_count * (1 + self.get_child_count(child))

        return total_bags


example_tree = BagGraph(example_bags)
assert set(example_tree.get_parents("shiny gold")) == set(("bright white", "muted yellow", "dark orange", "light red"))
assert example_tree.get_child_count("shiny gold") == 32

In [21]:
true_bags = [Bag.from_spec(spec) for spec in data]
print(f"Total Bags: {len(true_bags)}")
true_map = BagGraph(true_bags)
print(f"Total Containers: {len(set(true_map.get_parents('shiny gold')))}")

Total Bags: 594
Total Containers: 115


In [22]:
example2 = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
""".strip().splitlines()


example_bags2 = [Bag.from_spec(spec) for spec in example2]
example_map2 = BagGraph(example_bags2)
print(f"One shiny gold bag must contain {example_map2.get_child_count('shiny gold')} other bags")
assert example_map2.get_child_count("shiny gold") == 126

One shiny gold bag must contain 126 other bags


In [23]:
print(f"One shiny gold bag must contain {true_map.get_child_count('shiny gold')} other bags")

One shiny gold bag must contain 1250 other bags
